In [18]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import os
from pathlib import Path
import re
import pickle
# ver qtos o modelo apenas de ner acertaria
from transformers import AutoTokenizer, AutoModelForTokenClassification
import nltk    
from nltk import tokenize 
import torch
from transformers import BertTokenizer,BertForTokenClassification
import numpy as np
import json   
from importlib import reload  # Python 3.4+
import random
import model as mod
from model import BertForChunkClassification
from transformers import AdamW, BertConfig, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader
from transformers import AutoConfig

In [19]:
BATCH=10
print('BATCH:', BATCH)

BATCH: 10


In [41]:
def replaceWhiteSpaces(str):
    return re.sub('\s{2,}',' ',str)


def save_obj(name, obj):
    existeDir = os.path.exists('../obj')
    if not existeDir:
        os.makedirs('../obj')
    with open('../obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)

def load_obj(name):
    existeDir = os.path.exists('../obj')
    if not existeDir:
        os.makedirs('../obj')
    try:
        with open('../obj/' + name + '.pkl', 'rb') as f:
            return pickle.load(f)
    except:
        print('erro ao pegar obj')
        return None
    
def loadSentencesTest():
    # gabarito = dicSentences_new.pkl
    #dicSentences_train = load_obj('dic_sentencesTrain.pkl')
    #dicSentences_train[32]
    # gabarito = dicSentences_new.pkl
    print('Pegando sentencas de teste gabarito: dic_sentencesTest.pkl')
    dicSentences_new_test = load_obj('dic_sentencesTest')
    return dicSentences_new_test

def predictBERTNER_IO(sentencas, tipo_entidade):

    model=''
    if tipo_entidade == 'Tratamento':
        model = 'lisaterumi/portuguese-ner-biobertptclin-tratamento'
    elif tipo_entidade == 'Teste':
        model = 'lisaterumi/portuguese-ner-biobertptclin-teste'
    elif tipo_entidade == 'Anatomia':
        model = 'lisaterumi/portuguese-ner-biobertptclin-anatomia'
    elif tipo_entidade == 'Problema':
        model = 'lisaterumi/portuguese-ner-biobertptclin-problema'
    elif tipo_entidade == 'all':
        #model = 'lisaterumi/portuguese-ner-nestedclinbr-biobertpt-all'
        #model = 'lisaterumi/portuguese-ner-nestedclinbr-biobertpt-clin'
        model=r'C:\Users\lisat\OneDrive\jupyter notebook\NER-nestedclinbr\all'

    tokenizer = AutoTokenizer.from_pretrained(model)
    config = AutoConfig.from_pretrained(model)
    idx2tag = config.id2label
    #idx2tag = config.label2id 
    print('idx2tag:', idx2tag)
    model = AutoModelForTokenClassification.from_pretrained(model)

    predictedModel=[]
    all_tokens=[]
    
    for test_sentence in sentencas:
        #print('test_sentence:', test_sentence)
        tokenized_sentence = tokenizer.encode(test_sentence)
        input_ids = torch.tensor([tokenized_sentence])#.cuda()
        
        with torch.no_grad():
            output = model(input_ids)
        label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)
        
        # join bpe split tokens
        tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
        new_tokens, new_labels = [], []
        for token, label_idx in zip(tokens, label_indices[0]):
            if token.startswith("##"):
                new_tokens[-1] = new_tokens[-1] + token[2:]
            else:
                new_labels.append(label_idx)
                new_tokens.append(token)
            
        FinalLabelSentence = []
        FinalToken = []
        #print(len(new_tokens))
        #print(len(new_labels))
        for token, label in zip(new_tokens, new_labels):
            label = idx2tag[label]
            #label = str(label)
            if label == "O" or label == "X":
                FinalLabelSentence.append("O")
                FinalToken.append(token)
            else:
                FinalLabelSentence.append(label)
                FinalToken.append(token)
                
        predictedModel.append(FinalLabelSentence[1:-1]) # delete [SEP] and [CLS]
        all_tokens.append(FinalToken[1:-1])
                    
    return predictedModel, all_tokens

def getDicPredictions(tags, tokens):
    dic_predictions = {}
    num=-1
    #[tokens], [indices], tag
    for tags_sentence, tokens_sentence in zip(tags, tokens):
        num=num+1
        entidades = []
        tokens_frase = []
        entidade_anterior=0
        #print('---inicio sentenca:--')
        num_token=0
        indices_entidade = list()
        tokens_entidade = list()
        for tag, token in zip(tags_sentence, tokens_sentence):
            #print(token + ' ' + tag)
            tokens_frase.append([token, num_token])
            if tag!='O' and (entidade_anterior==tag or num==0):
                #entidades.append([num_token, tag])
                indices_entidade.append(num_token)
                #print('token:', token)
                tokens_entidade.append(token)
                entidade_anterior=tag
            #elif tag!='O': # nova entidade
            else:
                if entidade_anterior!='O' and len(tokens_entidade)>0: 
                    entidades.append([' '.join(tokens_entidade), indices_entidade, entidade_anterior])
                indices_entidade=list()
                tokens_entidade = list()
                if tag!='O': 
                    indices_entidade.append(num_token)
                    tokens_entidade.append(token)
                entidade_anterior=tag
            num_token=num_token+1
        #dic_predictions[num]=entidades
        dic_predictions[num]=[tokens_frase, entidades]
        entidades=list()
    return dic_predictions

def getDicPosTagger(dic_sentencesTrainDev):
    dicPostagger = load_obj('dic_postagger')
    allFrases = load_obj('allFrases')
    if dicPostagger==None or allFrases==None:
        print('Nao foi possivel recuperar obj, gerando dicPosTagger novamente')
        dicPostagger = {}
        allFrases=[]
        for key, value in dic_sentencesTrainDev.items():
            tokens=value[0]
            frase = [t[0] for t in tokens]
            frase = ' '.join(frase)
            allFrases.append(frase)
            #print(frase)

        #print(allFrases)
        doc = nlp_token_class(allFrases)
        #print(doc)
        for frase in doc:
            for d in frase:
                #print(d)
                pos = d['entity_group']
                #print(pos)
                token=d['word']
                if pos=='PREP+ART':
                    pos='ART'
                if pos=='NPROP':
                    pos='N'
                if 'ADV' in pos: # ADV-KS, ADV-KS-REL	
                    pos='ADV'
                if 'PRON' in pos: # PRO-KS	, PRO-KS-REL, PROPESS, PROPSUB
                    pos='PRON'
                if pos=='VAUX'or pos=='PCP': #participio
                    pos='V'
                dicPostagger[token] = pos    

        save_obj('dicPostagger', dicPostagger)
        save_obj('allFrases', allFrases)
    return dicPostagger, allFrases

def tipoPostaggerTokens(entidade_token, dicPostagger):
    postagger = ''
    for p in entidade_token:
        #print('p:', p)
        if p.lower() in dicPostagger.keys():
            postagger = postagger + '-' + dicPostagger.get(p.lower())
        else:
            #print('nao tem:', p)
            # se nao tem, considera N
            postagger = postagger + '-' + 'N'
    return postagger


def getListaPostaggerEntidades(dic_sentencesTrainDev, dicPosTagger):
    lista_postaggers_entidades = []
    for key, value in dic_sentencesTrainDev.items():
        #print('key:', key)
        entidades = value[1]
        for entidade in entidades:
            #print(entidade[0])
            pos_tagger=tipoPostaggerTokens(entidade[0].split(), dicPosTagger)
            if pos_tagger not in lista_postaggers_entidades:
                lista_postaggers_entidades.append(pos_tagger)
    return lista_postaggers_entidades

def getCombinacaoEntidadesSentence(dic_predictions, filtro_postagger, dicPosTagger, taxaDownsampling, lista_postaggers_entidades):
    #labels = {0:'O', 1:'Problema', 2:'Tratamento', 3:'Teste', 4:'Anatomia'}
    labels = {'O':0, 'Problema':1, 'Tratamento':2, 'Teste':3, 'Anatomia':4}
    num=0
    erro_corpus=0
    num_frases_sem_entidade=0
    lista_erro_corpus=list()
    combinacaoEntidadesPos = list()
    combinacaoEntidadesNeg = list()
    combinacaoEntidades = list()
    combinacaoEntidadesAll = list()
    pulando_termos_postagger = list()
    if filtro_postagger:
        print('Sentence Pairs - Com filtro-postagger')
    else:
        print('Sentence Pairs - Sem filtro-postagger')
    if taxaDownsampling>0:
        print('Sentence Pairs - Com taxa de Downsampling de ', taxaDownsampling)
    else:
        print('Sentence Pairs - Sem taxa de Downsampling')

    for key, value in dic_predictions.items():
        num=num+1
        tokens=value[0].copy()
        so_tokens = [t[0] for t in tokens]
        entidades=value[1].copy()
        num_positivas=0
        lista_indices = list()
        for entidade in entidades:
            erros_entidade = list()
            texto_entidade=entidade[0].strip()
            indices = entidade[1]
            tipo_entidade = entidade[2]
            frase = so_tokens.copy()
            inicio=indices[0]
            fim=indices[-1]
            lista_indices = [inicio, fim]
            #entidade_frase=frase[inicio:fim+1] # texto_entidade
            entidade_frase=texto_entidade
            #print('entidade_frase:', entidade_frase)
            #print('frase:', frase)
            #print('texto_entidade:', texto_entidade)
            if texto_entidade=='-' or texto_entidade=='=' or texto_entidade=='+' or texto_entidade==':' or texto_entidade==',' or texto_entidade=="'" or texto_entidade=='"' or texto_entidade=='.' or texto_entidade==';' or texto_entidade=='/' or texto_entidade=='(' or texto_entidade==')' or texto_entidade=='[' or texto_entidade==']':
                pass
            texto_entidade_comparar=texto_entidade.replace('/','').replace(')','').replace('(','').replace(']','').replace('[','').replace(',','').replace('.','').replace(';','').replace('-','').replace('+','').replace("'",'')
            texto_entidade_comparar = replaceWhiteSpaces(texto_entidade_comparar)
            texto_frase_comparar = ' '.join(frase[inicio:fim+1]).strip().replace('/','').replace(')','').replace('(','').replace(']','').replace('[','').replace(',','').replace('.','').replace(';','').replace('-','').replace('+','').replace("'",'')
            texto_frase_comparar = replaceWhiteSpaces(texto_frase_comparar)
            texto_entidade_comparar = texto_entidade_comparar.lower()
            texto_frase_comparar = texto_frase_comparar.lower()
            if (texto_entidade_comparar == texto_frase_comparar):
                num_positivas=num_positivas+1
                lista_indices_proc = [num for num in range(lista_indices[0], lista_indices[1]+1, 1)]
                combinacaoEntidadesPos.append([entidade_frase, ' '.join(frase).strip(), lista_indices_proc, labels[tipo_entidade]]) # apendando entidades reais
            else:
                print('erro, key:', key)
                erro_corpus=erro_corpus+1
                erros_entidade.append(indices)
                lista_erro_corpus.append([' '.join(frase).strip(), tipo_entidade, ' '.join(so_tokens), entidade])
        # agora, os negativos
        for entidade in entidades:
                indices = entidade[1]
                #print('indices:', indices)
                if indices in erros_entidade:
                    continue
                inicio=indices[0]
                fim=indices[-1]
                # agora, fazer a combinacao entre eles.. todas a seguir serão do tipo 'O'           
                for indice in indices:
                    for i in range(indice, fim+1):
                        # ver se nao tem antes
                        frase = so_tokens.copy()
                        termo = frase[indice:i+1]
                        frase_string=' '.join(termo).strip()
                        devePular = 0
                        if '.' in frase_string[-1:] or ',' in frase_string[-1:]  or ';' in frase_string[-1:] or '-' in frase_string[-1:]  or ':' in frase_string[-1:]  or '=' in frase_string[-1:]  or '/' in frase_string[-1:]  or '(' in frase_string[-1:]  or ')' in frase_string[-1:]  or '[' in frase_string[-1:]  or ']' in frase_string[-1:]  or ':' in frase_string[-1:]:
                            devePular=1
                        if '.' in frase_string[:1] or ',' in frase_string[:1]  or ';' in frase_string[:1] or '-' in frase_string[:1]  or ':' in frase_string[:1]  or '=' in frase_string[:1] or '/' in frase_string[:1]  or '(' in frase_string[:1]  or ')' in frase_string[:1] or '[' in frase_string[:1]  or ']' in frase_string[:1]  or ':' in frase_string[:1]:
                            devePular=1
                        if re.search("^[0-9]*mg", frase_string):
                            devePular=1
                            
                        if filtro_postagger==True:
                            pos_tagger_termo = tipoPostaggerTokens(termo, dicPosTagger)
                            if pos_tagger_termo not in lista_postaggers_entidades:
                                pulando_termos_postagger.append([termo, pos_tagger_termo])
                                devePular=1
                
                        tem_frase = 0
                        for frase_l in combinacaoEntidadesPos:
                            if frase_l[0] == frase_string:
                                tem_frase='1'
                                break
                        if tem_frase==0 and devePular==0:
                        #print('tem_frase:', tem_frase)
                        #if tem_frase==0:
                            lista_indices = [indice, len(frase_string.split())-1+indice]
                            lista_indices_proc = [num for num in range(lista_indices[0], lista_indices[1]+1, 1)]
                            combinacaoEntidadesNeg.append([frase_string, ' '.join(frase).strip(), lista_indices_proc, labels['O']])
                        
        # shuffle no combinacaoEntidades
        # taxaDownsampling, ex 2 para o dobro, 1 para mesma quantidade
        if len(combinacaoEntidadesPos)>0:
            if taxaDownsampling>0:
                combinacaoEntidadesNeg = combinacaoEntidadesNeg[:(num_positivas*taxaDownsampling)+num_positivas]
            random.shuffle(combinacaoEntidadesNeg)
        else:
            num_frases_sem_entidade = num_frases_sem_entidade+1
        if (num % 1000) ==0:
            print('key:', key)

        #print('combinacaoEntidadesNeg:',combinacaoEntidadesNeg)
        #combinacaoEntidades = combinacaoEntidades+combinacaoEntidadesPos+combinacaoEntidadesNeg
        combinacaoEntidades = combinacaoEntidadesPos+combinacaoEntidadesNeg
        combinacaoEntidadesPos=list()
        combinacaoEntidadesNeg=list()
        combinacaoEntidadesAll.append(combinacaoEntidades)
  
    print('erro_corpus:', erro_corpus)
    print('num_frases_sem_entidade:', num_frases_sem_entidade)
    print('len(combinacaoEntidadesAll:)',len(combinacaoEntidadesAll))
    
    return combinacaoEntidadesAll

def isEntidadeAninhada(a, b):
    inicio1=a[0]
    fim1=a[-1]
    #print(inicio1, fim1)
    inicio2=b[0]
    fim2=b[-1]
    #print(inicio2, fim2)
    if inicio1 >=inicio2 and fim1 <=fim2:
        return True
    else:
        return False

def posProcessamento(dic_predictions_sentence):
    dic_predictions_all = {}
    for key, value in dic_predictions_sentence.items():
        #dic_predictions_all[key] = value.copy()
        tokens = value[0].copy()
        listaEntidadesNer = list()
        listaIndicesNer=list()
        listaTipoNer=list()
        listaEntidadesSpan = dic_predictions_sentence[key][1]
        for entidadeNer in listaEntidadesSpan:
            listaIndicesNer.append(entidadeNer[1])
            listaTipoNer.append(entidadeNer[2])
        #print('listaIndicesNer:', listaIndicesNer)
        for entidadeSpan in listaEntidadesSpan:
            #print('entidadeSpan:', entidadeSpan)
            indices = entidadeSpan[1]
            #print('incluindo:', entidadeSpan)
            # só acrescenta se nao tem entidade mais externa do mesmo tipo
            # ver pelo indice
            isAninhada=False
            isMesmoTipo=False
            tipo=entidadeSpan[2]
            for indicesNer, tipoNer in zip(listaIndicesNer, listaTipoNer):
                isAninhada=False
                if indices!=indicesNer:
                    isAninhada = isEntidadeAninhada(indices, indicesNer) # é aninhada com alguma entidade?
                if isAninhada and tipoNer == tipo:
                    isMesmoTipo=True
                    #print('-----mesmo tipo!!')
                    #print('entidadeSpan:', entidadeSpan)
                    #print('tipoNer:', tipoNer)
                    #print('tipo:', tipo)
                    #print('indices:', indices)
                    #print('indicesNer:', indicesNer)
                    #print('indices!=indicesNer:', indices!=indicesNer)
                    break
            # ver pelo tipo tb
            if not (isAninhada and isMesmoTipo):
                #print('Não é aninhada ou tipos sao diferentes, incluindo:', entidadeSpan)
                #print('tipoNer:', tipoNer)
                #print('tipo:', tipo)
                listaEntidadesNer.append(entidadeSpan)
            else:
                #print('aninhada com mesmo tipo, nao entra:', entidadeSpan)
                #print('tipoNer:', tipoNer)
                #print('tipo:', tipo)
                pass               
        dic_predictions_all[key] = [tokens, listaEntidadesNer]
        #if key>15:
        #    break
    return dic_predictions_all

def AvalFinal(dicSentences_new_test, dic_predictions_all, BATCH):

    region_true_list, region_pred_list = list(), list() # labels
    region_true_count, region_pred_count = 0, 0 # contagem
    numErro1=0
    numErro2=0

    for i in range(0, BATCH, 1):
        if i<len(dicSentences_new_test):
            #print('\n---Label vs predicao--------')
            #print('frase:', dicSentences_new_test[i][0])
            #print('i:', i)
            #print(dicSentences_new_test[i][1])
            #print(dic_predictions_all[i][1])


            entidades_gabarito = dicSentences_new_test[i][1]
            try:
                entidades_preditas = dic_predictions_all[i][1]
            except:
                print('erro, caiu no except na AvalFinal:', str(i))
            #print('---entidades_gabarito--:', entidades_gabarito)
            #print('entidades_preditas:', entidades_preditas)
            for entidade_gabarito in entidades_gabarito:
                indices_gabarito = entidade_gabarito[1]
                tag_gabarito = entidade_gabarito[2]
                region_true_count=region_true_count+1
                region_true_list.append(tag_gabarito)
                # ver se previu essa entidade
                previu=0
                for entidade_predita in entidades_preditas:
                    indices_predita = entidade_predita[1]
                    tag_predita = entidade_predita[2]
                    if indices_predita == indices_gabarito:
                        region_pred_list.append(tag_predita)
                        previu=1
                        if tag_predita !='O':
                            region_pred_count=region_pred_count+1
                        break
                if previu==0:
                    numErro1=numErro1+1
                    region_pred_list.append('O')

            # agora o contrario, ver o q previu mas nao era

            for entidade_predita in entidades_preditas:
                indices_predita = entidade_predita[1]
                tag_predita = entidade_predita[2]
                # ver se a entidade prevista existe ou é FP
                existe=0
                for entidade_gabarito in entidades_gabarito:
                    indices_gabarito = entidade_gabarito[1]
                    if indices_predita == indices_gabarito:
                        existe=1
                        break
                if existe==0:
                    numErro2 = numErro2+1
                    region_true_list.append('O')
                    region_pred_list.append(tag_predita)

    #print('===resultados====')
    #print('region_true_list:', region_true_list)
    #print('region_pred_list:', region_pred_list)

    print('numErro1:', numErro1)
    print('numErro2:', numErro2)
    #print(classification_report(region_true_list, region_pred_list, labels=['O', 'PROTEIN', 'DNA', 'RNA', 'CELL_TYPE', 'CELL_LINE'], target_names=['O', 'PROTEIN', 'DNA', 'RNA', 'CELL_TYPE', 'CELL_LINE'], digits=6))
    print(classification_report(region_true_list, region_pred_list, digits=6))

    return region_true_list, region_pred_list

def getDicPredictionsSentence(combinacaoEntidadesAll_pred, dic_predictions):
    #labels = {0:'O', 1:'Problema', 2:'Tratamento', 3:'Teste', 4:'Anatomia'}
    labels = {0:'Problema', 1:'Tratamento', 2:'Teste', 3:'Anatomia'}
    num=-1
    dic_predictions_sentence={}
    entidades=list()
    numAcrescentou=0
    for frases in combinacaoEntidadesAll_pred:
        num=num+1
        for valor in frases:
            #print('num:', num)
            #print('valor:', valor)
            #print('valor[1]:', valor[1])
            tokens_entidade = valor[0]
            #frase = valor[1]
            indices = valor[2]
            tipo_previsto=valor[4]
            #print('tokens_entidade:', tokens_entidade)
            #print('frase:', frase)
            #print('indices:', indices)
            #print('tipo_previsto:', tipo_previsto)
            if tipo_previsto!=0:
                entidades.append([tokens_entidade, indices, labels[tipo_previsto]])
            # ver se entidade está na dic_prediction
        frase = dic_predictions[num].copy()[0]
        dic_predictions_sentence[num] = [frase, entidades]
        entidades = list()
                     
    return dic_predictions_sentence

def juntaPredictons(dic_predictions, dic_predictions_sentence):
    dic_predictions_all = {}
    for key, value in dic_predictions.items():
        #dic_predictions_all[key] = value.copy()
        tokens = value[0].copy()
        listaEntidadesNer = value[1].copy()
        listaIndicesNer=list()
        listaTipoNer=list()
        for entidadeNer in listaEntidadesNer:
            listaIndicesNer.append(entidadeNer[1])
            listaTipoNer.append(entidadeNer[2])
        #print('listaIndicesNer:', listaIndicesNer)
        listaEntidadesSpan = dic_predictions_sentence[key][1]
        for entidadeSpan in listaEntidadesSpan:
            indices = entidadeSpan[1]
            if indices in listaIndicesNer:
                #print('ja tem, pulando')
                pass
            else:
                #print('incluindo:', entidadeSpan)
                # só acrescenta se nao tem entidade mais externa do mesmo tipo
                # ver pelo indice
                isAninhada=False
                isMesmoTipo=False
                tipo=entidadeSpan[2]
                for indicesNer, tipoNer in zip(listaIndicesNer, listaTipoNer):
                    isAninhada = isEntidadeAninhada(indices, indicesNer) # é aninhada com alguma entidade?
                    if isAninhada and tipoNer == tipo:
                        isMesmoTipo=True
                        #print('-----mesmo tipo!!')
                        #print('entidadeSpan:', entidadeSpan)
                        #print('tipoNer:', tipoNer)
                        #print('tipo:', tipo)
                        break
                # ver pelo tipo tb
                if not (isAninhada and isMesmoTipo):
                    #print('Não é aninhada ou tipos sao diferentes, incluindo:', entidadeSpan)
                    #print('tipoNer:', tipoNer)
                    #print('tipo:', tipo)
                    listaEntidadesNer.append(entidadeSpan)
                else:
                    #print('aninhada com mesmo tipo, nao entra:', entidadeSpan)
                    #print('tipoNer:', tipoNer)
                    #print('tipo:', tipo)
                    pass               
        dic_predictions_all[key] = [tokens, listaEntidadesNer]
        #if key>15:
        #    break
    return dic_predictions_all

In [21]:
dicSentences_new_test = loadSentencesTest()
print(len(dicSentences_new_test))
dicSentences_new_test = {k: v for k, v in dicSentences_new_test.items() if k<BATCH}
print(dicSentences_new_test[0])
#print(dicSentences_new_test[27])
print('numero de sentencas no total:', len(dicSentences_new_test))

sentences=list()
for key, value in dicSentences_new_test.items():
    if key<BATCH:
        tokens = value[0]
        tokens = [tok[0] for tok in tokens]
        sentences.append(' '.join(tokens).strip())
#print(sentences[0])

tags, tokens = predictBERTNER_IO(sentences, 'all')
dic_predictions = getDicPredictions(tags, tokens)
#print(dic_predictions[0])
print('len(dic_predictions):', len(dic_predictions))
#print(dic_predictions[9])
        
print('len(dicSentences_new_test):', len(dicSentences_new_test))
print('len(dic_predictions):', len(dic_predictions))


Pegando sentencas de teste gabarito: dic_sentencesTest.pkl
506
[[['Lucas', 0, 43], [',', 1, 48], ['74', 2, 50], ['anos', 3, 53], ['.', 4, 57]], []]
numero de sentencas no total: 10
idx2tag: {0: 'Teste', 1: 'Anatomia', 2: 'O', 3: 'Problema', 4: 'Tratamento', 5: '<pad>'}
len(dic_predictions): 10
len(dicSentences_new_test): 10
len(dic_predictions): 10


### Testando modelo de sentence-pais
Só positivos com pos processamento
1) com filtro postagger

2) sem filtro postagger

In [24]:
from simpletransformers.classification import (
    ClassificationModel, ClassificationArgs
)
import pandas as pd
import logging

print('-----Avaliando só com modelo de Sentence Pairs:-----')
model = ClassificationModel('bert', 'lisaterumi/sentence_pairs_nested_positivos', use_cuda=False)


-----Avaliando só com modelo de Sentence Pairs:-----


Downloading:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/679M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/560 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [29]:
#df = pd.read_csv(r"..\preProcessamento\sentence-pairs-filtro\sentence_pairs.test", delimiter='\t', header=0, names=['text_b', 'text_a', 'labels'])
#df = df.dropna(axis=0, how='any')
#print('Número de sentenças de test: {:,}\n'.format(df.shape[0]))
#df=df[:50]
#test_df = df
#print(test_df[:3])
#lista=list()
#for index, row in test_df.iterrows():
#    lista.append([row['text_b'], row['text_a']])
#predictions, raw_outputs = model.predict(
#  lista
#)
dic_sentencesTrainDev = load_obj('dic_sentencesTrainDev')
dicPosTagger, _ = getDicPosTagger(dic_sentencesTrainDev)
lista_postaggers_entidades = getListaPostaggerEntidades(dic_predictions, dicPosTagger)
# para gerar arquivo de predicoes (com as tags <e1>)
print('Primeiro, com filtro postagger:')
combinacaoEntidadesAll = getCombinacaoEntidadesSentence(dic_predictions, True, dicPosTagger, 0, lista_postaggers_entidades)



Primeiro, com filtro postagger:
Sentence Pairs - Com filtro-postagger
Sentence Pairs - Sem taxa de Downsampling
erro_corpus: 0
num_frases_sem_entidade: 2
len(combinacaoEntidadesAll:) 10


In [30]:
%%capture 
# para suprimir output
#combinacaoEntidadesAll = f.load_obj('combinacaoEntidadesAll_'+str(BATCH))
print('Chamando predict')
pred_region_labels = list()
for key, combinacao in enumerate(combinacaoEntidadesAll):
    if key<BATCH:    
        if len(combinacao)>0:
            lista = [l[0:2] for l in combinacao]
            predictions, _ = model.predict(lista) 
            pred_region_labels.append(predictions)
            for comb, label in zip(combinacao, predictions):
                comb.append(label)
#f.save_obj('combinacaoEntidadesAllSentenceComLabel1_'+str(BATCH), combinacaoEntidadesAll)


In [40]:
dic_predictions_sentence = getDicPredictionsSentence(combinacaoEntidadesAll, dic_predictions)
dic_predictions_sentence[1]

[[['Em', 0],
  ['acompanhamento', 1],
  ['no', 2],
  ['ambualtorio', 3],
  ['há', 4],
  ['5', 5],
  ['anos', 6],
  ['por', 7],
  ['FA', 8],
  [',', 9],
  ['uso', 10],
  ['de', 11],
  ['marevan', 12],
  ['5mg', 13],
  ['1', 14],
  ['x', 15],
  ['ao', 16],
  ['dia', 17],
  ['.', 18]],
 [['marevan 5mg', [12, 13], 'Tratamento'], ['marevan', [12], 'Tratamento']]]

In [36]:
# avaliar por regiao.. 
print('-----Avaliando modelo com filtro:-----')

region_true_list, region_pred_list = AvalFinal(dicSentences_new_test, dic_predictions_sentence, BATCH)
print(confusion_matrix(region_true_list, region_pred_list))

print('Com pos processamento')

dic_predictions_all = posProcessamento(dic_predictions_sentence)
#dic_predictions_all
region_true_list, region_pred_list = AvalFinal(dicSentences_new_test, dic_predictions_all, BATCH)
print(confusion_matrix(region_true_list, region_pred_list))

print('Juntando - pos processamento + dic_predictions (primeiro nivel -> ner)')

dic_predictions_all = posProcessamento(dic_predictions_sentence)
dic_predictions_all = juntaPredictons(dic_predictions, dic_predictions_all)
#dic_predictions_all
region_true_list, region_pred_list = AvalFinal(dicSentences_new_test, dic_predictions_all, BATCH)
print(confusion_matrix(region_true_list, region_pred_list))


-----Avaliando modelo com filtro:-----
numErro1: 16
numErro2: 5
              precision    recall  f1-score   support

    Anatomia   1.000000  1.000000  1.000000         1
           O   0.000000  0.000000  0.000000         5
    Problema   0.000000  0.000000  0.000000        15
       Teste   1.000000  0.666667  0.800000         3
  Tratamento   0.687500  1.000000  0.814815        11

    accuracy                       0.400000        35
   macro avg   0.537500  0.533333  0.522963        35
weighted avg   0.330357  0.400000  0.353228        35

[[ 1  0  0  0  0]
 [ 0  0  0  0  5]
 [ 0 15  0  0  0]
 [ 0  1  0  2  0]
 [ 0  0  0  0 11]]
Com pos processamento
numErro1: 16
numErro2: 0
              precision    recall  f1-score   support

    Anatomia   1.000000  1.000000  1.000000         1
           O   0.000000  0.000000  0.000000         0
    Problema   0.000000  0.000000  0.000000        15
       Teste   1.000000  0.666667  0.800000         3
  Tratamento   1.000000  1.000000  1.0

C:\Users\lisat\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lisat\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
combinacaoEntidadesAll = getCombinacaoEntidadesSentence(dic_predictions, False, '', 0, '')
print('-----Avaliando modelo sem filtro:-----')
print('Chamando predict')


In [ ]:
%%capture 
pred_region_labels = list()
for key, combinacao in enumerate(combinacaoEntidadesAll):
    if key<BATCH:    
        if len(combinacao)>0:
            lista = [l[0:2] for l in combinacao]
            predictions, _ = model.predict(lista) 
            pred_region_labels.append(predictions)
            for comb, label in zip(combinacao, predictions):
                comb.append(label)

In [ ]:
dic_predictions_sentence = getDicPredictionsSentence(combinacaoEntidadesAll, dic_predictions)
region_true_list, region_pred_list = AvalFinal(dicSentences_new_test, dic_predictions_sentence, BATCH)
print(confusion_matrix(region_true_list, region_pred_list))

print('com pos processamento')

dic_predictions_all = posProcessamento(dic_predictions_sentence)
#dic_predictions_all
region_true_list, region_pred_list = AvalFinal(dicSentences_new_test, dic_predictions_all, BATCH)
print(confusion_matrix(region_true_list, region_pred_list))


print('Juntando - pos processamento + dic_predictions (primeiro nivel -> ner)')

dic_predictions_all = posProcessamento(dic_predictions_sentence)
dic_predictions_all = juntaPredictons(dic_predictions, dic_predictions_all)
#dic_predictions_all
region_true_list, region_pred_list = AvalFinal(dicSentences_new_test, dic_predictions_all, BATCH)
print(confusion_matrix(region_true_list, region_pred_list))
